<a href="https://colab.research.google.com/github/arcursino/tcc_mba/blob/main/TCC_Ariana_Cursino_Modelagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalações

In [ ]:
!pip install bambi

In [ ]:
!pip install --upgrade scikit-learn


# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import is_numeric_dtype, is_object_dtype, is_categorical_dtype
import pymc as pm
import arviz as az
import statsmodels.api as sm
from statsmodels.genmod.bayes_mixed_glm import BinomialBayesMixedGLM
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, classification_report, mean_absolute_error, mean_squared_error
import bambi as bmb
import arviz as az

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)


In [ ]:
# Import machine learning methods
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 200

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Pre-Processo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TCC_MBA/alignment_synthetic_data.csv')

In [ ]:
def remover_colunas_numericas_constantes(df):
    """
    Identifica e remove colunas numéricas de um DataFrame pandas
    que possuem um valor constante em todas as linhas.

    Args:
        df (pd.DataFrame): O DataFrame de entrada.

    Returns:
        tuple: Uma tupla contendo:
            - pd.DataFrame: O DataFrame com as colunas constantes removidas.
            - list: Uma lista com os nomes das colunas que foram removidas.
    """
    print("Iniciando a avaliação das colunas...")

    # 1. Selecionar apenas as colunas que são de tipos numéricos
    # Usamos np.number para incluir todos os tipos numéricos (int, float, etc.)
    colunas_numericas = df.select_dtypes(include=np.number).columns
    print(f"Total de colunas numéricas encontradas: {len(colunas_numericas)}")

    # Lista para armazenar os nomes das colunas a serem removidas
    colunas_para_remover = []

    # 2. Iterar sobre as colunas numéricas e verificar se são constantes
    print("Verificando colunas numéricas para constância...")
    for coluna in colunas_numericas:
        # Verifica o número de valores únicos na coluna
        # Se for 1, significa que todos os valores são iguais
        if df[coluna].nunique() == 1:
            colunas_para_remover.append(coluna)
            print(f"  - Coluna '{coluna}' é constante e será removida.")
        # else: # Opcional: para ver quais colunas numéricas NÃO são constantes
        #     print(f"  - Coluna '{coluna}' não é constante.")


    # 3. Remover as colunas identificadas do DataFrame original
    if colunas_para_remover:
        print(f"\nRemovendo {len(colunas_para_remover)} coluna(s) constante(s)...")
        df_limpo = df.drop(columns=colunas_para_remover)
        print("Remoção concluída.")
    else:
        print("\nNenhuma coluna numérica constante encontrada. O DataFrame permanece inalterado.")
        df_limpo = df.copy() # Retorna uma cópia para manter a consistência

    print("\nAvaliação e remoção finalizadas.")

    # Retorna o DataFrame limpo e a lista de colunas removidas
    return df_limpo, colunas_para_remover

In [ ]:
df_limpo, colunas_removidas = remover_colunas_numericas_constantes(df)

# Modelos

## Train e Test

In [ ]:
df_train = df_limpo[['cell_id', 'Vehicle_Type', 'Warranty',
       'Rear_Thrust_Angle_Final_Value', 'RIDE_HEIGHT_FA_DIFF',
       'RIDE_HEIGHT_FL', 'RIDE_HEIGHT_FL_INITIAL', 'RIDE_HEIGHT_FR',
       'RIDE_HEIGHT_RA_DIFF', 'RIDE_HEIGHT_RL', 'RIDE_HEIGHT_RR',
       'Steering_Wheel_Angle_Final_Value', 'Steering_Wheel_Angle_Preset_Value',
       'Front_Sum_Toe_Final_Value', 'TOE_FA_TOTAL_INITIAL', 'L_F_Toe',
       'TOE_FL_INITIAL', 'R_F_Toe', 'TOE_FR_INITIAL', 'TOE_RA_TOTAL',
       'TOE_RA_TOTAL_INITIAL', 'L_R_Toe', 'TOE_RL_INITIAL', 'R_R_Toe',
       'TOE_RR_INITIAL', 'CabStyle','Engine', 'RearWheels', 'SteeringGear',
        'Suspension', 'Tire', 'Transmission', 'Wheel', 'LaneDeparture', 'MY']].copy()

In [ ]:
object_cols = df_train.select_dtypes(include='object').columns
for col in object_cols:
    df_train[col] = df_train[col].astype('category')

In [ ]:
X_train, X_test = train_test_split(df_train, random_state=42, test_size=0.3)

In [ ]:
id_and_categorical_cols = ['cell_id', 'Vehicle_Type', 'Warranty','CabStyle', 'Engine',
                           'RearWheels', 'SteeringGear', 'Suspension', 'Tire',
                           'Transmission', 'Wheel','LaneDeparture', 'MY']
numeric_cols = []

for col in X_train.columns:
  if is_numeric_dtype(df_limpo[col]):
    numeric_cols.append(col)

# Separar as colunas
X_train_numerical = X_train[numeric_cols]
X_train_ids_and_cats = X_train[id_and_categorical_cols]

# Separar as colunas
X_test_numerical = X_test[numeric_cols]
X_test_ids_and_cats = X_test[id_and_categorical_cols]

# --- Passo 3: Aplique a escala APENAS às colunas numéricas ---
scaler = StandardScaler() # Ou seu scaler de preferência
X_train_scaled_numerical_array = scaler.fit_transform(X_train_numerical)
X_test_scaled_numerical_array = scaler.fit_transform(X_test_numerical)

# --- Passo 4: Converta o array escalado de volta para DataFrame e combine com as colunas originais ---
# É crucial nomear as colunas do array escalado para poder combiná-las
X_train_scaled_numerical_df = pd.DataFrame(X_train_scaled_numerical_array, columns=numeric_cols, index=X_train.index)
X_test_scaled_numerical_df = pd.DataFrame(X_test_scaled_numerical_array, columns=numeric_cols, index=X_test.index)

# Combine os dados numéricos escalados com as colunas de ID/categóricas originais
# Use o index para garantir que as linhas correspondam
X_train_processed = pd.concat([X_train_scaled_numerical_df, X_train_ids_and_cats], axis=1)
X_test_processed = pd.concat([X_test_scaled_numerical_df, X_test_ids_and_cats], axis=1)

In [ ]:
target = X_train_processed.copy()
target_teste = X_test_processed.copy()

## Baseline

In [ ]:
formula = 'Warranty ~ Rear_Thrust_Angle_Final_Value + RIDE_HEIGHT_FA_DIFF + RIDE_HEIGHT_FL_INITIAL +\
           RIDE_HEIGHT_FR + RIDE_HEIGHT_RA_DIFF + RIDE_HEIGHT_RL + RIDE_HEIGHT_RR + Steering_Wheel_Angle_Final_Value +\
           Steering_Wheel_Angle_Preset_Value + Front_Sum_Toe_Final_Value + TOE_FA_TOTAL_INITIAL + L_F_Toe +\
           TOE_FL_INITIAL + R_F_Toe + TOE_FR_INITIAL + TOE_RA_TOTAL + TOE_RA_TOTAL_INITIAL + L_R_Toe + TOE_RL_INITIAL +\
           R_R_Toe + TOE_RR_INITIAL +  cell_id+ Vehicle_Type Drive_+ Engine_+ Cab_+ CabStyle + Engine + RearWheels +\
           SteeringGear + Suspension + Tire + Transmission + Wheel + wheelbase_broadcast + LaneDeparture + MY'

modelo = bmb.Model(formula, target, family="bernoulli")
modelo.build()
modelo.graph()

In [ ]:
data1 = modelo.fit(draws=2000, tune=1000, chains=4,target_accept=0.95,
                    random_seed=123,
                    idata_kwargs={"log_likelihood": True})

In [ ]:
pm.summary(data1, hdi_prob=0.95, round_to=3)

In [ ]:
ll_pointwise = data1.log_likelihood["Warranty"]

In [ ]:
# 2. somar sobre cada observação → log-likelihood total por amostra
    # Usar '__obs__' em vez de 'observation'
ll_total = ll_pointwise.sum(dim="__obs__")         # dims: ("chain", "draw")

    # 3. Se quiser empilhar cadeias e amostras em um só eixo:
ll_total_flat = ll_total.stack(sample=("chain", "draw")).values
print(np.mean(ll_total_flat))

In [ ]:
loo = az.loo(data1)
print(f"LOO: {loo}")

In [ ]:
modelo.plot_priors()

## VIF

In [ ]:
for columns in df_train[object_cols]:
  print(df_train[columns].value_counts())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import matplotlib.pyplot as plt

def process_and_analyze_features(df_input: pd.DataFrame):
    """
    Realiza o pré-processamento de features numéricas e categóricas,
    calcula o VIF (Variance Inflation Factor) para identificar multicolinearidade
    e exibe um heatmap de correlação.

    Args:
        df_input (pd.DataFrame): O DataFrame de entrada contendo as features
                                  e a coluna 'Warranty'.

    Returns:
        tuple: Uma tupla contendo:
               - X_processed (pd.DataFrame): O DataFrame de features processadas.
               - vif_data (pd.DataFrame): O DataFrame com os resultados do VIF.
    """
    # Separar features (X) e target (y)
    X = df_input.drop('Warranty', axis=1)
    y = df_input['Warranty']

    # --- Identificar features numéricas e categóricas (usando suas listas) ---
    numerical_features_to_scale = X.select_dtypes(include=np.number).columns.tolist()
    categorical_features = X.select_dtypes(include='category').columns.tolist()

    # --- Filtrar X para ter apenas as features que você listou ---
    all_expected_features = numerical_features_to_scale + categorical_features
    X = X[all_expected_features]

    # --- 2. Pré-processar Features Numéricas (Standard Scaling) ---
    scaler = StandardScaler()
    X_numerical_scaled_array = scaler.fit_transform(X[numerical_features_to_scale])

    # Converta o array de volta para DataFrame usando os nomes corretos
    X_numerical_processed = pd.DataFrame(X_numerical_scaled_array, columns=numerical_features_to_scale, index=X.index)

    # --- 3. Pré-processar Features Categóricas (pd.get_dummies) ---
    X_categorical_processed = pd.get_dummies(X[categorical_features], columns=categorical_features, drop_first=True)

    # --- 4. Combinar as features processadas (numéricas escaladas e categóricas dummy) ---
    X_processed = pd.concat([X_numerical_processed, X_categorical_processed], axis=1)

    # --- VERIFICAÇÕES DE SEGURANÇA E CONVERSÃO DE TIPO (CRÍTICO) ---

    # 1. Converte todas as colunas para tipo numérico, coercing erros para NaN
    X_processed = X_processed.apply(pd.to_numeric, errors='coerce')

    # 2. **NOVO PASSO CRÍTICO**: Força o DataFrame inteiro para float64
    # Isso garante que todas as colunas tenham um dtype numérico adequado para operações NumPy
    X_processed = X_processed.astype(np.float64)

    # 3. Verifica e lida com NaN (se algum foi introduzido por 'coerce' ou por problemas do scaler)
    if X_processed.isnull().sum().sum() > 0:
        print("AVISO: NaNs encontrados nos dados processados após conversão. Preenchendo com a média da coluna.")
        # Preencher NaNs com a média da coluna
        X_processed = X_processed.fillna(X_processed.mean())
        # Se uma coluna inteira for NaN (e.g., se to_numeric resultou em NaN para toda a coluna),
        # a média também será NaN. Preenche com 0 nesses casos.
        X_processed = X_processed.fillna(0)

    # 4. Verifica e lida com Infinitos (se algum foi introduzido pelo scaler)
    if np.isinf(X_processed.values).sum() > 0:
        print("AVISO: Infs encontrados nos dados processados. Substituindo por NaN e depois preenchendo com 0.")
        X_processed = X_processed.replace([np.inf, -np.inf], np.nan).fillna(0)

    #print(f"Dtypes dos dados processados DEPOIS da limpeza:\n{X_processed.dtypes}\n")

    # --- 5. Calcular o VIF para cada feature ---
    # Adiciona uma constante (intercepto) ao DataFrame para o cálculo do VIF
    X_processed_with_const = sm.add_constant(X_processed, has_constant='add')

    vif_data = pd.DataFrame()
    vif_data["feature"] = X_processed_with_const.columns
    # Não precisa mais do .astype(np.float64) aqui, pois já foi feito acima
    vif_data["VIF"] = [variance_inflation_factor(X_processed_with_const.values, i)
                       for i in range(X_processed_with_const.shape[1])]

    # Remover a linha da constante do VIF
    vif_data = vif_data[vif_data['feature'] != 'const']

    # Classificar para facilitar a visualização
    vif_data = vif_data.sort_values(by="VIF", ascending=False).reset_index(drop=True)

    print("\nResultados do VIF:")
    print(vif_data)

    # --- 6. Interpretação (Exemplo) ---
    print("\n--- Interpretação ---")
    high_vif_features = vif_data[vif_data['VIF'] >= 3]
    if not high_vif_features.empty:
        print("\nFeatures com alta multicolinearidade (VIF >= 3):")
        print(high_vif_features)
        print("\nConsidere remover ou combinar essas features para melhorar a estabilidade do modelo.")
    else:
        print("\nNenhuma feature com alta multicolinearidade (VIF >= 3) encontrada.")

    # --- 7. Outros métodos visuais (complementares) ---
    plt.figure(figsize=(15, 12))
    sns.heatmap(X_processed.corr(), cmap='coolwarm', linewidths=.1, linecolor='lightgray', square=True)
    plt.title('Matriz de Correlação das Features Processadas')
    plt.show()

    return X_processed, vif_data

In [ ]:
check_multi = process_and_analyze_features(df_train)

In [ ]:
df_train.columns

In [ ]:
filtered_columns_for_vif = [
    'Warranty', # variável target
    # Vamos listar apenas as preditoras para o X.

    # Features Numéricas
    'Rear_Thrust_Angle_Final_Value',
    'RIDE_HEIGHT_FA_DIFF',
    'RIDE_HEIGHT_RA_DIFF',
    'RIDE_HEIGHT_RL',
    'RIDE_HEIGHT_RR',
    'Steering_Wheel_Angle_Final_Value',
    'Steering_Wheel_Angle_Preset_Value',
    'TOE_RA_TOTAL', # Mantida, mas será crucial observar seu VIF novamente

    # Features Categóricas (serão expandidas por pd.get_dummies)
    'cell_id',
    'CabStyle',
    'Engine',
    'RearWheels',
    'SteeringGear',
    'Suspension', # Mantida, pd.get_dummies(drop_first=True) deve resolver o VIF alto de seus dummies
    'Tire',
    'Transmission',
    'Wheel',
    'LaneDeparture',
    'Vehicle_Type', # Mantida (efeito aleatório no modelo, mas tratada como fixa para VIF)
    'MY'            # Mantida (efeito aleatório no modelo, mas tratada como fixa para VIF)
]

X_for_vif = df_train[filtered_columns_for_vif].copy()

In [ ]:
X_for_vif.columns

In [ ]:
check_multi = process_and_analyze_features(X_for_vif)

In [ ]:
X_for_vif.columns

## Base sem multicolineriedade

In [ ]:
X_for_vif.columns

In [ ]:
target_vif = target[['Warranty', 'Rear_Thrust_Angle_Final_Value', 'RIDE_HEIGHT_FA_DIFF',
       'RIDE_HEIGHT_RA_DIFF', 'RIDE_HEIGHT_RL', 'RIDE_HEIGHT_RR',
       'Steering_Wheel_Angle_Final_Value', 'Steering_Wheel_Angle_Preset_Value',
       'TOE_RA_TOTAL', 'cell_id', 'CabStyle', 'Engine', 'RearWheels',
       'SteeringGear', 'Suspension', 'Tire', 'Transmission', 'Wheel',
       'LaneDeparture', 'Vehicle_Type', 'MY']]

In [ ]:
formula_vif2 = 'Warranty ~ Rear_Thrust_Angle_Final_Value +  RIDE_HEIGHT_FA_DIFF +\
        RIDE_HEIGHT_RA_DIFF +  RIDE_HEIGHT_RL +  RIDE_HEIGHT_RR +\
        Steering_Wheel_Angle_Final_Value +  Steering_Wheel_Angle_Preset_Value +\
        TOE_RA_TOTAL +  cell_id +  CabStyle +  Engine +  RearWheels +\
        SteeringGear +  Suspension +  Tire +  Transmission +  Wheel +\
        LaneDeparture +  Vehicle_Type +  MY'

modelo_sem_vif2 = bmb.Model(formula_vif2, target_vif, family="bernoulli")

In [ ]:
#print(modelo)

In [ ]:
modelo_sem_vif2.build()
modelo_sem_vif2.graph()

In [ ]:
data1_sem_vif2 = modelo_sem_vif2.fit(draws=2000, tune=1000, chains=4,target_accept=0.95,
                    random_seed=123,
                    idata_kwargs={"log_likelihood": True})

In [ ]:
pm.summary(data1_sem_vif2, hdi_prob=0.95, round_to=3)

In [ ]:
ll_pointwise_sem_vif2 = data1_sem_vif2.log_likelihood["Warranty"]

In [ ]:
#z.plot_trace(data1)

In [ ]:
# 2. somar sobre cada observação → log-likelihood total por amostra
    # Usar '__obs__' em vez de 'observation'
ll_total_sem_vif2 = ll_pointwise_sem_vif2.sum(dim="__obs__")         # dims: ("chain", "draw")

    # 3. Se quiser empilhar cadeias e amostras em um só eixo:
ll_total_flat_sem_vif2 = ll_total_sem_vif2.stack(sample=("chain", "draw")).values
print(np.mean(ll_total_flat_sem_vif2))



In [ ]:
loo = az.loo(data1_sem_vif2)
print(f"LOO: {loo}")

## Multinivel VT

In [ ]:
formula_multi = 'Warranty ~ TOE_RA_TOTAL + RIDE_HEIGHT_RA_DIFF + ' \
          'LaneDeparture + SteeringGear + Wheel + ' \
          '(1|Vehicle_Type)'

modelo_multi = bmb.Model(formula_multi, target, family="bernoulli")

In [ ]:
#print(modelo)

In [ ]:
modelo_multi.build()
modelo_multi.graph()

In [ ]:
data1_multi = modelo_multi.fit(draws=2000, tune=1000, chains=4,target_accept=0.99,
                    random_seed=123,
                    idata_kwargs={"log_likelihood": True})

In [ ]:
pm.summary(data1_multi, hdi_prob=0.95, round_to=3)

In [ ]:
ll_pointwise_multi = data1_multi.log_likelihood["Warranty"]

In [ ]:
#z.plot_trace(data1)

In [ ]:
# 2. somar sobre cada observação → log-likelihood total por amostra
    # Usar '__obs__' em vez de 'observation'
ll_total_multi = ll_pointwise_multi.sum(dim="__obs__")         # dims: ("chain", "draw")

    # 3. Se quiser empilhar cadeias e amostras em um só eixo:
ll_total_flat_multi = ll_total_multi.stack(sample=("chain", "draw")).values
print(np.mean(ll_total_flat_multi))



In [ ]:
loo = az.loo(data1_multi)
print(f"LOO: {loo}")

## Multinivel Pit

In [ ]:
formula_multi_2 = 'Warranty ~ TOE_RA_TOTAL + RIDE_HEIGHT_RA_DIFF + ' \
          'LaneDeparture + SteeringGear + Wheel + ' \
          '(1|cell_id)'

modelo_multi_2 = bmb.Model(formula_multi_2, target, family="bernoulli")

In [ ]:
#print(modelo)

In [ ]:
modelo_multi_2.build()
modelo_multi_2.graph()

In [ ]:
data1_multi_2 = modelo_multi_2.fit(draws=2000, tune=1000, chains=4,target_accept=0.999,
                    random_seed=123, max_treedepth=25,
                    idata_kwargs={"log_likelihood": True})

In [ ]:
pm.summary(data1_multi_2, hdi_prob=0.95, round_to=3)

In [ ]:
ll_pointwise_multi_2 = data1_multi_2.log_likelihood["Warranty"]

In [ ]:
#z.plot_trace(data1)

In [ ]:
# 2. somar sobre cada observação → log-likelihood total por amostra
    # Usar '__obs__' em vez de 'observation'
ll_total_multi_2 = ll_pointwise_multi_2.sum(dim="__obs__")         # dims: ("chain", "draw")

    # 3. Se quiser empilhar cadeias e amostras em um só eixo:
ll_total_flat_multi_2 = ll_total_multi_2.stack(sample=("chain", "draw")).values
print(np.mean(ll_total_flat_multi_2))



In [ ]:
loo = az.loo(data1_multi_2)
print(f"LOO: {loo}")

## Multinivel SG

In [ ]:
formula_multi_sg = 'Warranty ~ RIDE_HEIGHT_RL + Steering_Wheel_Angle_Final_Value + RIDE_HEIGHT_RA_DIFF + \
                  RIDE_HEIGHT_RR + Wheel + (1|SteeringGear)'

modelo_multi_sg = bmb.Model(formula_multi_sg, target, family="bernoulli")

In [ ]:
#print(modelo)

In [ ]:
modelo_multi_sg.build()
modelo_multi_sg.graph()

In [ ]:
data1_multi_sg = modelo_multi_sg.fit(draws=2000, tune=1000, chains=4,target_accept=0.999,
                    random_seed=123, max_treedepth=25,
                    idata_kwargs={"log_likelihood": True})

In [ ]:
pm.summary(data1_multi_sg, hdi_prob=0.95, round_to=3)

In [ ]:
ll_pointwise_multi_sg = data1_multi_sg.log_likelihood["Warranty"]

In [ ]:
#z.plot_trace(data1)

In [ ]:
# 2. somar sobre cada observação → log-likelihood total por amostra
    # Usar '__obs__' em vez de 'observation'
ll_total_multi_sg = ll_pointwise_multi_sg.sum(dim="__obs__")         # dims: ("chain", "draw")

    # 3. Se quiser empilhar cadeias e amostras em um só eixo:
ll_total_flat_multi_sg = ll_total_multi_sg.stack(sample=("chain", "draw")).values
print(np.mean(ll_total_flat_multi_sg))



In [ ]:
loo = az.loo(data1_multi_sg)
print(f"LOO: {loo}")

## Multinivel Wheel

In [ ]:
formula_wheel = 'Warranty ~ TOE_RA_TOTAL + RIDE_HEIGHT_RA_DIFF + SteeringGear + (1|Wheel)'

modelo_wheel = bmb.Model(formula_wheel, target, family="bernoulli")

In [ ]:
#print(modelo)

In [ ]:
modelo_wheel.build()
modelo_wheel.graph()

In [ ]:
data1_wheel = modelo_wheel.fit(draws=2000, tune=1000, chains=4,target_accept=0.95,
                    random_seed=123,
                    idata_kwargs={"log_likelihood": True})

In [ ]:
pm.summary(data1_wheel, hdi_prob=0.95, round_to=3)

In [ ]:
ll_pointwise_wheel = data1_wheel.log_likelihood["Warranty"]

In [ ]:
#z.plot_trace(data1)

In [ ]:
# 2. somar sobre cada observação → log-likelihood total por amostra
    # Usar '__obs__' em vez de 'observation'
ll_total_wheel = ll_pointwise_wheel.sum(dim="__obs__")         # dims: ("chain", "draw")

    # 3. Se quiser empilhar cadeias e amostras em um só eixo:
ll_total_flat_wheel = ll_total_wheel.stack(sample=("chain", "draw")).values
print(np.mean(ll_total_flat_wheel))



In [ ]:
loo = az.loo(data1_wheel)
print(f"LOO: {loo}")

## Multinivel Wheel 2

In [ ]:
formula_wheel = 'Warranty ~ TOE_RA_TOTAL + (1|Wheel)'

modelo_wheel = bmb.Model(formula_wheel, target, family="bernoulli")

In [ ]:
#print(modelo)

In [ ]:
modelo_wheel.build()
modelo_wheel.graph()

In [ ]:
data1_wheel = modelo_wheel.fit(draws=2000, tune=1000, chains=4,target_accept=0.999,
                    random_seed=123,max_treedepth=25,
                    idata_kwargs={"log_likelihood": True})

In [ ]:
pm.summary(data1_wheel, hdi_prob=0.95, round_to=3)

In [ ]:
ll_pointwise_wheel = data1_wheel.log_likelihood["Warranty"]

In [ ]:
#z.plot_trace(data1)

In [ ]:
# 2. somar sobre cada observação → log-likelihood total por amostra
    # Usar '__obs__' em vez de 'observation'
ll_total_wheel = ll_pointwise_wheel.sum(dim="__obs__")         # dims: ("chain", "draw")

    # 3. Se quiser empilhar cadeias e amostras em um só eixo:
ll_total_flat_wheel = ll_total_wheel.stack(sample=("chain", "draw")).values
print(np.mean(ll_total_flat_wheel))



In [ ]:
loo = az.loo(data1_wheel)
print(f"LOO: {loo}")

## Classico Toe e Wheel

In [ ]:
formula_wheel_toe = 'Warranty ~ TOE_RA_TOTAL + Wheel'

modelo_wheel_toe = bmb.Model(formula_wheel_toe, target, family="bernoulli")

In [ ]:
#print(modelo)

In [ ]:
modelo_wheel_toe.build()
modelo_wheel_toe.graph()

In [ ]:
data1_wheel_toe = modelo_wheel_toe.fit(draws=2000, tune=1000, chains=4,target_accept=0.95,
                    random_seed=123,
                    idata_kwargs={"log_likelihood": True})

In [ ]:
pm.summary(data1_wheel_toe, hdi_prob=0.95, round_to=3)

In [ ]:
ll_pointwise_wheel_toe = data1_wheel_toe.log_likelihood["Warranty"]

In [ ]:
#z.plot_trace(data1)

In [ ]:
# 2. somar sobre cada observação → log-likelihood total por amostra
    # Usar '__obs__' em vez de 'observation'
ll_total_wheel_toe = ll_pointwise_wheel_toe.sum(dim="__obs__")         # dims: ("chain", "draw")

    # 3. Se quiser empilhar cadeias e amostras em um só eixo:
ll_total_flat_wheel_toe = ll_total_wheel_toe.stack(sample=("chain", "draw")).values
print(np.mean(ll_total_flat_wheel_toe))



In [ ]:
loo = az.loo(data1_wheel_toe)
print(f"LOO: {loo}")